In [1]:
import argparse
import os
import pandas as pd
from datetime import datetime
from functools import partial
import numpy as np
import joblib
import tensorflow as tf
from tensorflow import keras
import sys

assert tf.__version__ >= "2.0"

# To make the output stable across runs
tf.random.set_seed(42)

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE_PER_REPLICA = 64
!nvidia-smi


Sat Dec 10 09:14:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    27W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
def create_model(kernel_size):
    DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=kernel_size,
                            activation='relu',
                            padding="valid")

    return keras.models.Sequential([
        DefaultConv2D(filters=256, input_shape=[34, 121, 1]),
        keras.layers.BatchNormalization(),


        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),


        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),

        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),


        DefaultConv2D(filters=32),
        keras.layers.BatchNormalization(),

        keras.layers.Flatten(),

        keras.layers.Dense(units=256, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),

        keras.layers.Dense(units=34, activation='softmax'),
    ])

In [3]:
dataset_path = '/content/drive/MyDrive/Mahjong_data/discard_data.csv'
logs_path = '/content/drive/MyDrive/Training_log '
cnn_path = '/content/drive/MyDrive/CNN_model'


In [4]:
def data_format(data34:int):
    # print(data34)
    res = np.array([0 for _ in range(34)])
    for i in range(34):
        res[i] =  1 if ((1<<(34-i-1)) & data34 )!= 0 else 0
    return res

In [5]:
data_all = pd.read_csv(dataset_path,header = None)

In [6]:
data_all.shape

(1634987, 122)

In [7]:
def modify_dataframe(df):
    np_df = df.to_numpy()
    new_data = []
    for i in range(np_df.shape[0]):
        tmp = np.array([[0]*34]*np_df.shape[1])
        for j in range(np_df.shape[1]):
            tmp[j] = data_format(np_df[i][j])
        new_data.append(tmp.reshape(34,(np_df.shape[1]),1))
    return np.array(new_data)
        

In [8]:
X_train = modify_dataframe(data_all.iloc[:100000,:-1])
y_train = modify_dataframe(data_all.iloc[:100000, -1:]).reshape(100000,34)
X_dev = modify_dataframe(data_all.iloc[150000:160000, :-1])
y_dev = modify_dataframe(data_all.iloc[150000:160000, -1:]).reshape(10000,34)

In [13]:
print('X_train shape:', X_train.shape)
print('X_dev.shape:', X_dev.shape)
print('y_train shape:', y_train.shape)
print('y_dev.shape:', y_dev.shape)

X_train shape: (100000, 34, 121, 1)
X_dev.shape: (10000, 34, 121, 1)
y_train shape: (100000, 34)
y_dev.shape: (10000, 34)



In [11]:
import psutil 
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

53.71026201851995

In [12]:
del data_all
psutil.virtual_memory().available * 100 / psutil.virtual_memory().total

52.569332495464465

In [14]:
def train(custom_name,kernel_size = (3,1), custom_epochs = 25):
    
    tf.keras.backend.clear_session()

    strategy = tf.distribute.MirroredStrategy()
    num_of_gpus = strategy.num_replicas_in_sync
    print('Number of devices:', num_of_gpus)
    print()

    with strategy.scope():
        model = create_model(kernel_size)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        print(model.summary())
        print()

    # Train the neural network
    model_name = 'discard_cnn_' + str(kernel_size[0]) + str(kernel_size[1]) + custom_name
    log_dir = os.path.join(logs_path + '/' + model_name + '/tensorboard_logs',
                           datetime.now().strftime("%Y%m%d-%H%M%S"))
    checkpoint_prefix = os.path.join(logs_path + '/' + model_name
                                     + '/checkpoints',
                                     'checkpoint_{epoch}')

    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=log_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                           save_best_only=True,
                                           save_weights_only=True,
                                           period=10)
    ]

    BATCH_SIZE = BATCH_SIZE_PER_REPLICA * num_of_gpus
    history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=custom_epochs,
                        validation_data=(X_dev, y_dev),
                        callbacks=callbacks)

    # Save the neural network
    model.save(os.path.join(cnn_path, model_name + '.h5'))

    eval_train = model.evaluate(X_train, y_train)
    print('final training loss:', eval_train[0])
    print('final training accuracy:', eval_train[1])
    eval_dev = model.evaluate(X_dev, y_dev)
    print('final dev loss:', eval_dev[0])
    print('final dev accuracy:', eval_dev[1])

In [ ]:
train("_Final_10w_data_5conv31",(3,1),4)

Number of devices: 1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 121, 256)      1024      
                                                                 
 batch_normalization (BatchN  (None, 32, 121, 256)     1024      
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 121, 256)      196864    
                                                                 
 batch_normalization_1 (Batc  (None, 30, 121, 256)     1024      
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 121, 256)      196864    
                                                                 
 batch_normalization_2 (Batc  (Non

None

Epoch 1/4
1563/1563 [==============================] - 550s 343ms/step - loss: 2.3829 - accuracy: 0.3216 - val_loss: 2.6437 - val_accuracy: 0.3328
Epoch 2/4
1563/1563 [==============================] - 535s 342ms/step - loss: 1.9874 - accuracy: 0.4082 - val_loss: 1.8632 - val_accuracy: 0.4306
Epoch 3/4
1563/1563 [==============================] - 534s 341ms/step - loss: 1.8980 - accuracy: 0.4326 - val_loss: 1.8128 - val_accuracy: 0.4466
Epoch 4/4
1563/1563 [==============================] - 533s 341ms/step - loss: 1.8345 - accuracy: 0.4480 - val_loss: 1.8169 - val_accuracy: 0.4463
